# Cálculo de ruta óptima entre puntos 

## 1. Carga de librerias y ficheros de entrada

Se cargan inicialmente las librerías y ficheros necesarios para el análisis. En este caso se parte del fichero de distancias entre puntos ya filtrado y limpiado previamente.

In [4]:
import requests
import json
from datetime import datetime
from itertools import combinations
import pandas as pd
import numpy as np
import time
import folium
from folium.plugins import HeatMap
import os
import joblib
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
os.chdir("drive/My Drive")
df_dist = pd.read_csv('df_grafo_desc_fin.csv',delimiter=",")

In [7]:
df_dist_espejo = df_dist.iloc[:, [6,7,8,9,10,11,0,1,2,3,4,5,12,13,16,17,14,15]]

In [8]:
df_dist_espejo = df_dist_espejo.rename({'punto_fin': 'punto_ini', 'denom_fin': 'denom_ini','tipo_carg_fin': 'tipo_carg_ini', 'provincia_fin': 'provincia_ini','municipio_fin': 'municipio_ini', 'tipo_vehic_fin': 'tipo_vehic_ini','lat_fin': 'lat_ini', 'lon_fin': 'lon_ini',
                       'punto_ini': 'punto_fin', 'denom_ini': 'denom_fin','tipo_carg_ini': 'tipo_carg_fin', 'provincia_ini': 'provincia_fin','municipio_ini': 'municipio_fin', 'tipo_vehic_ini': 'tipo_vehic_fin','lat_ini': 'lat_fin', 'lon_ini': 'lon_fin'}, axis=1)

In [9]:
df_dist = pd.concat([df_dist, df_dist_espejo])

## 2. Estimación de consumo energético para trayectos

Antes de proceder a la estimación de la ruta óptima, es necesario añadir los datos de consumo energético durante cada uno de los trayectos entre todos los puntos del grafo. (Recordemos que cada registro del dataset cargado es un trayecto entre dos puntos cualesquiera del grafo)

## 2.1 Preparación de las variables de entrada y carga del modelo

En primer lugar se cargará el modelo ya generado y ajustado previamente con los datos de trayectos de vehículos disponibles públicamente a través del repositorio de datos abiertos del gobierno australiano. Fuente: https://data.gov.au/data/dataset/smart-grid-smart-city-electric-vehicle-trial-data 

In [10]:
EV_rf = joblib.load("random_forest_EV.joblib")

Para predecir los datos de consumo, es necesario proporcionar las variables utilizadas durante la construcción del algoritmo.

- **Velocidad**:Velocidad promedio durante del trayecto en km/h.

- **Distancia**: distancia de conducción entre dos puntos medida en km.

- **Tiempo encendido faros delanteros**: tiempo en minutos durante el cual los faros delanteros del vehículo han estado conectados durante el trayecto.

- **Tiempo encendido aire acondicionado**: tiempo en minutos durante el cual el aire acondicionado del vehículo ha estado conectados durante el trayecto.

### 2.1.1 Distancia

La variable distancia medida en Km ha sido extraida directamente a través de la API de Google Maps y no requiere ninguna transformación adicional.

### 2.1.2 Velocidad

La velocidad promedio del trayecto es factible estimarla a partir de los datos proporcionados por la API de Google Maps. En el presente dataset de trayectos se dispone tanto de la distancia entre puntos en km como la duración del trayecto en segundos dependiendo de la velocidad máxima de las vías de circulación y el estado del tráfico. Dado que los datos se extrajeron un día entresemana no festivo, a las 19:00, se asumirá que los datos de tiempo de trayecto son lo suficientemente representativos de un trayecto tipo. 

Para calcular la velocidad promedio en km/h se procederá de la siguiente forma:

# $v = \frac{dist \,}{duracion/3600}$

In [11]:
df_dist["vel_promedio"] = df_dist["distancia"]/(df_dist["duracion"]/3600)

### 2.1.3 Tiempo encendido faros delanteros

En una primera aproximación se tendrán en cuenta trayectos diurnos y por tanto un tiempo de uso promedio de faros delanteros de un 15 % de la duración total del trayecto (posibles garajes o túneles durante el trayecto o en caso de lluvia o niebla)

In [12]:
df_dist["dur_headlights"] = (df_dist["duracion"]*0.15/60)

### 2.1.4 Tiempo encendido aire acondicionado

Se asumirá como día estándar para la utilización del aire acondicionado, un día primaveral con temperaturas moderadas, con un uso bajo de climatización interior. Se define un porcentaje de uso del aire acondicionado equivalente a una cuarta parte del trayecto (25 %).

In [13]:
df_dist["dur_clim"] = (df_dist["duracion"]*0.25/60)

In [14]:
df_dist.head()

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,denom_fin,tipo_carg_fin,provincia_fin,municipio_fin,tipo_vehic_fin,distancia,duracion,lat_ini,lon_ini,lat_fin,lon_fin,vel_promedio,dur_headlights,dur_clim
0,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,164.0,6091,41.321165,2.027702,40.807620,0.520070,96.929897,15.2275,25.379167
1,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.38455, 2.1376713)",HOTEL NH NUMANCIA,NORMAL,Barcelona,Barcelona,NaN,15.1,1150,41.321165,2.027702,41.384550,2.137671,47.269565,2.8750,4.791667
2,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.394009000000004, 2.115265)",B:SM 20 - Marques de Mulhacen,NORMAL,Barcelona,Barcelona,cotxe i moto,13.8,933,41.321165,2.027702,41.394009,2.115265,53.247588,2.3325,3.887500
3,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.4047, 2.1896)",B:SM 24 - Ona Glòries,NORMAL,Barcelona,Barcelona,cotxe,22.5,1644,41.321165,2.027702,41.404700,2.189600,49.270073,4.1100,6.850000
4,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.66519, 1.86011)",EdRsR Sant Vicenç de Castellet (AC22kW),semiRAPID,Barcelona,Sant Vicenç de Castellet,NaN,52.1,2512,41.321165,2.027702,41.665190,1.860110,74.665605,6.2800,10.466667


### 2.1.5 Dataset de entrada del modelo

Para el dataset de entrada al modelo, se utilizarán las variables anteriormente mencionadas. Se eliminarán previamente todas los registros con duración igual a 0, ya que pueden introducir valores infinitos de velocidad que provocarían un error en las predicciones. Los nombres de las variables para este dataset de entrada x, se cambiarán a los nombres de los campos del conjunto de datos original, ya que el modelo fue construido con esta configuración.

In [15]:
df_dist = df_dist[df_dist['duracion']>0]

In [16]:
#Selección de variables de entrada al modelo.
x = df_dist.iloc[:, [18,20,19,12]]

In [17]:
x.rename(columns = {'distancia':'TOTAL_TRIP_DISTANCE','dur_clim':'AC_ON_DURATION','dur_headlights':'HEADLAMP_ON_DURATION','vel_promedio':'TEL_AVG_VELOCITY'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## 2.2 Estimación del consumo energético

A continuación se estima el consumo energético para cada trayecto con las variables de entrada en las unidades definidas y posteriormente se añade como columna al dataset general en estudio.

In [18]:
y = EV_rf.predict(x)

In [19]:
df_dist["cons_energ"] = y

In [20]:
df_dist

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,denom_fin,tipo_carg_fin,provincia_fin,municipio_fin,tipo_vehic_fin,distancia,duracion,lat_ini,lon_ini,lat_fin,lon_fin,vel_promedio,dur_headlights,dur_clim,cons_energ
0,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,164.0,6091,41.321165,2.027702,40.807620,0.520070,96.929897,15.2275,25.379167,64.079472
1,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.38455, 2.1376713)",HOTEL NH NUMANCIA,NORMAL,Barcelona,Barcelona,NaN,15.1,1150,41.321165,2.027702,41.384550,2.137671,47.269565,2.8750,4.791667,14.642674
2,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.394009000000004, 2.115265)",B:SM 20 - Marques de Mulhacen,NORMAL,Barcelona,Barcelona,cotxe i moto,13.8,933,41.321165,2.027702,41.394009,2.115265,53.247588,2.3325,3.887500,15.294823
3,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.4047, 2.1896)",B:SM 24 - Ona Glòries,NORMAL,Barcelona,Barcelona,cotxe,22.5,1644,41.321165,2.027702,41.404700,2.189600,49.270073,4.1100,6.850000,22.628382
4,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.66519, 1.86011)",EdRsR Sant Vicenç de Castellet (AC22kW),semiRAPID,Barcelona,Sant Vicenç de Castellet,NaN,52.1,2512,41.321165,2.027702,41.665190,1.860110,74.665605,6.2800,10.466667,56.458578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70493,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(42.426809999999996, 1.9292200000000002)",EdRR Puigcerdà (DC50kW AC43kW) PIRVEC-20,RAPID,Girona,Puigcerdà,NaN,151.0,6972,41.372299,2.153134,42.426810,1.929220,77.969019,17.4300,29.050000,67.533215
70494,"(41.56064, 2.0059)",Aparcament Plaça Progrés,NORMAL,Barcelona,Terrassa,cotxe i moto,"(42.426809999999996, 1.9292200000000002)",EdRR Puigcerdà (DC50kW AC43kW) PIRVEC-20,RAPID,Girona,Puigcerdà,NaN,124.0,5497,41.560640,2.005900,42.426810,1.929220,81.207932,13.7425,22.904167,66.950116
70495,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,52.0,2635,41.372299,2.153134,41.613690,2.345490,71.043643,6.5875,10.979167,56.614270
70496,"(41.56064, 2.0059)",Aparcament Plaça Progrés,NORMAL,Barcelona,Terrassa,cotxe i moto,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,40.8,1960,41.560640,2.005900,41.613690,2.345490,74.938776,4.9000,8.166667,42.887716


En este punto es conveniente introducir una corrección sobre el consumo debido al avance en la tecnología del vehículo eléctrico en los últimos años.

Las estimaciones de consumo del modelo, han sido generadas en base a trayectos reales realizados por un vehículo eléctrico Mitsubishi iMiEV, Modelo 2010. Este vehículo además de ser más antigua su tecnología, también tiene un tamaño más reducido, siendo su batería de menor capacidad.

Dado que los consumos entre trayectos se han generado en porcentaje de carga total de la batería, de cara a este estudio es necesario comprobar que este consumo relativo es similar a los modelos actuales de vehículos presentes en el mercado español de características similares. Atendiendo a ev-database, la capacidad real de la batería del Mitsubishi iMiEV es de  145 Wh/km. Si comparamos con uno de los modelos actuales más vendidos en el mercado español en el año 2021, el Fiat 500e,su consumo real promedio en un entorno mixto urbano/autopista es de 138 Wh/km.

https://ev-database.org/car/1285/Fiat-500e-Hatchback-42-kWh

https://ev-database.org/car/1029/Mitsubishi-i-MiEV

Dada la similitud entre consumos promedio, se mantendrán los valores iniciales.


Revisando la autonomía real del modelo seleccionado, el Mitsubishi iMiEV modelo 2010 tiene una autonomía real de un promedio de 100 km para un entorno combinado autopista y urbano. Se eliminarán todos los trayectos de distancia superior a 90 km.

In [21]:
#df_dist["cons_energ"] = df_dist["cons_energ"]*(14.5/37.3)*(138/145)

In [22]:
df_dist = df_dist[df_dist["distancia"]<90]
df_dist = df_dist[df_dist["tipo_carg_ini"]!="NORMAL"]
df_dist = df_dist[df_dist["tipo_carg_fin"]!="NORMAL"]
df_dist = df_dist[df_dist["tipo_carg_ini"]!="FORA DE SERVEI"]
df_dist = df_dist[df_dist["tipo_carg_fin"]!="FORA DE SERVEI"]

In [23]:
len(list(set([*df_dist["denom_ini"], *df_dist["denom_fin"]])))

183

In [24]:
df_dist

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,denom_fin,tipo_carg_fin,provincia_fin,municipio_fin,tipo_vehic_fin,distancia,duracion,lat_ini,lon_ini,lat_fin,lon_fin,vel_promedio,dur_headlights,dur_clim,cons_energ
390,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,"(41.15188, 1.2143700000000002)",EdRsR Constantí (AC22kW),semiRAPID i NORMAL,Tarragona,Constantí,cotxe i moto,86.4,3473,40.807620,0.520070,41.151880,1.214370,89.559459,8.6825,14.470833,72.605373
391,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,"(40.99197, 0.9214700000000001)",EdRR Vandellòs i Hospitalet de l'Infant (DC50k...,RAPID,Tarragona,Vandellòs i l'Hospitalet i l'infant,NaN,51.3,2159,40.807620,0.520070,40.991970,0.921470,85.539602,5.3975,8.995833,57.841226
434,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,"(41.08598, 1.03674)",EdRR CEPSA Cambrils (DC50kW AC43kW+AC22kW),RAPID i semiRAPID,Tarragona,Cambrils,NaN,64.3,2569,40.807620,0.520070,41.085980,1.036740,90.105099,6.4225,10.704167,77.180981
456,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,"(41.11432, 1.15586)",IBIL ES REPSOL-Vilaseca (DC50kW AC43kW),RAPID,Tarragona,Vilaseca,cotxe,76.2,2968,40.807620,0.520070,41.114320,1.155860,92.425876,7.4200,12.366667,75.684150
459,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,"(40.88558, 0.80385)",EdRsR L'Ametlla de Mar (AC22kW),semiRAPID i NORMAL,Tarragona,L'Ametlla de Mar,NaN,34.3,1617,40.807620,0.520070,40.885580,0.803850,76.363636,4.0425,6.737500,32.958483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70465,"(41.508264000000004, 2.138665)",IBIL-CC Baricentro (11kW),semiRAPID i NORMAL,Barcelona,Barberà del Vallès,cotxe i moto,"(41.884890000000006, 2.8747599999999998)",EdRsR Cassà de la Selva (AC22kW),NaN,Girona,Cassà de la Selva,NaN,85.7,3471,41.508264,2.138665,41.884890,2.874760,88.885048,8.6775,14.462500,72.835587
70467,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.884890000000006, 2.8747599999999998)",EdRsR Cassà de la Selva (AC22kW),NaN,Girona,Cassà de la Selva,NaN,62.8,2581,41.613690,2.345490,41.884890,2.874760,87.593956,6.4525,10.754167,79.291470
70472,"(41.508264000000004, 2.138665)",IBIL-CC Baricentro (11kW),semiRAPID i NORMAL,Barcelona,Barberà del Vallès,cotxe i moto,"(41.5379, 2.45285)",EdRsR Marítim Mataró (DC25kW AC25kW),semiRAPID,Barcelona,Mataró,NaN,34.9,1777,41.508264,2.138665,41.537900,2.452850,70.703433,4.4425,7.404167,33.868589
70474,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.5379, 2.45285)",EdRsR Marítim Mataró (DC25kW AC25kW),semiRAPID,Barcelona,Mataró,NaN,21.0,1167,41.613690,2.345490,41.537900,2.452850,64.781491,2.9175,4.862500,20.784942


In [25]:
import sys
 
class Graph(object):
    def __init__(self, nodes, init_graph):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, init_graph)
        
    def construct_graph(self, nodes, init_graph):
        '''
        Este método se asegura de que el grafo es simétrico, por tanto si existe una ruta de A a B con distancia n, tiene que existir un camino de B a A con distancia n.
        '''
        graph = {}
        for node in nodes:
            graph[node] = {}
        
        graph.update(init_graph)
        
        for node, edges in graph.items():
            for adjacent_node, value in edges.items():
                if graph[adjacent_node].get(node, False) == False:
                    graph[adjacent_node][node] = value
                    
        return graph
    
    def get_nodes(self):
        "Devuelve los nodos del grafo."
        return self.nodes
    
    def get_outgoing_edges(self, node):
        "Devuelve los vecinos de un nodo."
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)
        return connections
    
    def value(self, node1, node2):
        "Devuelve la distancia entre nodos."
        return self.graph[node1][node2]

In [26]:
def print_result(previous_nodes, shortest_path, start_node, target_node):
    path = []
    consumo = []
    duracion = []
    tipo_carga = []
    nodo_carga = []
    node = target_node
    
    while node != start_node:
        path.append(node)
        consumo.append(graph.value(node, previous_nodes[node])[1])
        duracion.append(graph.value(node, previous_nodes[node])[2])
        node = previous_nodes[node]
        
    # Add the start node manually
    path.append(start_node)
    
    SOC_status = 100
    n_cargas = 0
    dur = 0
    cont = 0
    lista_ruta = list(reversed(path))
    for gasto in consumo:
      if SOC_status < 25:
        SOC_status = 100
        n_cargas = n_cargas + 1
        tipo_carga.append(df_dist[df_dist["denom_ini"]==node]["tipo_carg_ini"].values[0])
        nodo_carga.append(lista_ruta[cont])
      SOC_status = SOC_status - gasto
      cont = cont + 1

    for tem in duracion:
      dur = dur + tem 
    for carga in tipo_carga:
      if ("RAPID" not in carga) and ("semiRAPID" in carga):
        dur = dur + (3*3600)  
      elif ("RAPID" in carga) and ("semiRAPID" not in carga):
        dur = dur + (1*3600)
      elif ("RAPID" in carga) and ("semiRAPID" in carga):
        dur = dur + (1*3600)
      elif carga == "superRAPID":
        dur = dur + (0.5*3600)

    dur = dur/3600
    print("La ruta óptima para alcanzar el punto destino es: {}.".format(shortest_path[target_node]))
    print(" -> ".join(reversed(path)))
    print(tipo_carga)
    print("Durante el trayecto se han realizado {n} cargas y queda un {s} por ciento de batería.".format(n = n_cargas,s = SOC_status))
    print("\n")
    print("Se ha realizado una carga en los siguientes puntos {p}".format(p = nodo_carga))
    print("El tipo de carga ha sido {t}".format(t = tipo_carga))
    print("La duración total del trayecto incluyendo tiempo de carga ha sido de {d} horas".format(d = dur))
    #print("El estado de carga final de la batería es: {}.".format(soc))
    print(consumo)

In [27]:
def dijkstra_algorithm_EV(graph, start_node):
    unvisited_nodes = list(graph.get_nodes())
 
    # We'll use this dict to save the cost of visiting each node and update it as we move along the graph   
    shortest_path = {}
    # We'll use this dict to save the shortest known path to a node found so far
    previous_nodes = {}
    # We'll use max_value to initialize the "infinity" value of the unvisited nodes   
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = [max_value,max_value,max_value]
    # However, we initialize the starting node's value with 0   
    shortest_path[start_node] = [0,0,0]
    
    # The algorithm executes until we visit all nodes
    while unvisited_nodes:
        # The code block below finds the node with the lowest score
        current_min_node = None

        for node in unvisited_nodes: # Iterate over the nodes
            if current_min_node == None:
                current_min_node = node

            elif (shortest_path[node][0] < shortest_path[current_min_node][0]):
                current_min_node = node

        # The code block below retrieves the current node's neighbors and updates their distances
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node][0] + graph.value(current_min_node, neighbor)[0]
            tentative_value_SOC = shortest_path[current_min_node][1] + graph.value(current_min_node, neighbor)[1]
            tentative_value_duration = shortest_path[current_min_node][2] + graph.value(current_min_node, neighbor)[2]
            if tentative_value < shortest_path[neighbor][0]:
                shortest_path[neighbor][0] = tentative_value
                shortest_path[neighbor][1] = tentative_value_SOC
                shortest_path[neighbor][2] = tentative_value_duration
                # We also update the best path to the current node
                previous_nodes[neighbor] = current_min_node
        # After visiting its neighbors, we mark the node as "visited"
        unvisited_nodes.remove(current_min_node)
    return previous_nodes,shortest_path
#fuente: https://www.udacity.com/blog/2021/10/implementing-dijkstras-algorithm-in-python.html con modificaciones

In [28]:
nodes = df_dist["denom_ini"].unique()
 
init_graph = {}
for node in nodes:
    init_graph[node] = {}

for index, row in df_dist.iterrows():
    init_graph[row["denom_ini"]][row["denom_fin"]] = [row["distancia"],row["cons_energ"],row["duracion"]]

In [29]:
graph = Graph(nodes, init_graph)

In [30]:
previous_nodes, shortest_path = dijkstra_algorithm_EV(graph=graph, start_node="EdRsR Tortosa (AC22kW) PIRVEC-24")

In [31]:
print_result(previous_nodes, shortest_path, start_node="EdRsR Tortosa (AC22kW) PIRVEC-24", target_node="EdRsR Balaguer (AC11kW)")

La ruta óptima para alcanzar el punto destino es: [153.1, 169.18683463454175, 8031].
EdRsR Tortosa (AC22kW) PIRVEC-24 -> EdRsR Ascó-Unió (AC22kW) -> EdRsR Lleida (AC22kW) -> EdRsR Balaguer (AC11kW)
['semiRAPID']
Durante el trayecto se han realizado 1 cargas y queda un 28.75017745386863 por ciento de batería.


Se ha realizado una carga en los siguientes puntos ['EdRsR Lleida (AC22kW)']
El tipo de carga ha sido ['semiRAPID']
La duración total del trayecto incluyendo tiempo de carga ha sido de 2.2308333333333334 horas
[26.623156770746796, 71.3138553176636, 71.24982254613137]


In [32]:
#df_dist[(df_dist["denom_ini"] == "77 SABA Rambla Catalunya") & (df_dist["denom_fin"] == "B:SM 24 - Ona Glòries")]
#df_dist[(df_dist["denom_ini"] == "EdRsR Tortosa (AC22kW) PIRVEC-24")]
#previous_nodes["Boï-Taull Resort"]